    87. Design the Frontend

Ardit shows the process of him drawing the "design-frontend.jpg" file in our images folder.

The app has two screens: the Camera Screen and the Image Screen.

__Camera Screen__

The Camera Screen displays the webcam of the user. The webcam will activate when the user presses the "Start" button. The second "Capture" button will capture a frame of the active webcam and convert it to an image. At the same time, the button will open the second Image Screen and display the captured screen.

__Image Screen__

The Image Screen displays the captured image in a large widget. Below it is a large button for "Create Link". Clicking this button will activate a widget with a link for the image. Below this initially invisible widget are two different buttons: "Copy" and "Open". The Copy button will copy the image link to the user's clipboard. The Open button will open the image link in the user's browser.

    88. Designing the Object Types

Ardit simply shows the process of him creating the design.txt document.

    89. Creating the Empty Classes

Just like we learned in the previous Webcam app, we're going to introduce some boiler plate Python/Kivy code to the MainApp file.

In [ ]:
from kivy.app import App
from kivy.uix.screenmanager import ScreenManager, Screen
from kivy.lang import Builder

Builder.load_file('frontend.kv')


class CameraScreen(Screen):

    def start(self):
        pass

    def stop(self):
        pass

    def capture(self):
        pass


class FileSharer: # I need to go back into the last module of the Flatmates Bill Sharing App to find this code explanation.

    def __init__(self, filepath, api_key="AViVqp7suSQWWEdrl6hf9z"):
        self.filepath = filepath
        self.api_key = api_key

    # def share(self):
    #     client = Client(self.api_key)
    #     new_filelink = client.upload(filepath=self.filepath)
    #     return new_filelink.url
    

class ImageScren(Screen):
    pass


class RootWidget(ScreenManager):
    pass

class MainApp(App):

    def build(self):
        return RootWidget()

MainApp().run()

Despite being listed as a class object in the design document, we are already going to define the functionality of the webcam in the CameraScreen class so we do not need to create a Webcam class of its own.

    90. Creating the Camera Screen

We're going to encounter an error in this module regarding a missing package. This is the "opencsv-python" package which allows us to activate the camera remotely through python code.

In [ ]:
# New frontend code
Builder.load_string("""
<CameraScreen>:
    GridLayout:
        cols: 1
        Camera: # Kivy has it's own inherent camera widget
            resolution: (640, 480)
            play: False # Stating False here will make it so that the camera will not start automatically when the code is run.
        Button:
            text: 'Start Camera'
        Button:
            text: 'Capture Image'

<RootWidget>:
    CameraScreen:
        id: camera_screen
        name: 'camera_screen'
""")    

    91. Starting the Camera

In [ ]:
# Updating frontend to include a button press action on the camera screen, as well as ID the camera widget.
Builder.load_string("""
<CameraScreen>:
    GridLayout:
        cols: 1
        Camera:
            id: camera
            resolution: (640, 480)
            play: False 
        Button:
            text: 'Start Camera'
            on_press: root.start()
        Button:
            text: 'Capture Image'

<RootWidget>:
    CameraScreen:
        id: camera_screen
        name: 'camera_screen'
""")    

In [ ]:
# Creating a method which attaches a command to the Camera Widget through the widget ID.
class CameraScreen(Screen):
    
    def start(self):
        self.ids.camera.play = True

Above, we simply use the "play" command inherent to the Camera method in the kivy library. By setting it equal to "True", we allow the camera to play when pressing the Start Camera button linked to the "start" method.

    92. Stopping the Camera

We will be writing code which will allow us to stop the camera feed after we press the "Start Camera" button a second time. Do make an intelligent frontend in this fashion, we will use conditionals in the kivy file.

In [ ]:
Builder.load_string("""
<CameraScreen>:
    GridLayout:
        cols: 1
        Camera:
            id: camera
            resolution: (640, 480)
            play: False 
        Button:
            text: 'Start Camera'
            on_press: root.start() if root.ids.camera.play == False else root.stop()
        Button:
            text: 'Capture Image'

<RootWidget>:
    CameraScreen:
        id: camera_screen
        name: 'camera_screen'
""")

The new code on the on_press action states that the action will only run on the method if the camera is already *off*. If the the camera is anything else other than off (i.e. on), the command will run the "stop" action instead.

However, if we want the name of the button to change to "Stop Camera" while the camera is running, we need to design some more frontend code of the camera widget.

In [ ]:
Builder.load_string("""
<CameraScreen>:
    GridLayout:
        cols: 1
        Camera:
            id: camera
            resolution: (640, 480)
            play: False 
        Button:
            id: camera_button
            text: 'Start Camera'
            on_press: root.start() if root.ids.camera.play == False else root.stop()
        Button:
            text: 'Capture Image'

<RootWidget>:
    CameraScreen:
        id: camera_screen
        name: 'camera_screen'
""")

My initial instinct was to create a conditional on the Button text line which describes what the button should say under certain circumstances. However, we can make this a bit cleaner and more efficient if we simple run a similar command attached to a widget ID in the "start" and "stop" methods of the CameraScreen class.

In [ ]:
class CameraScreen(Screen):
    
    def start(self):
        self.ids.camera.play = True
        self.ids.camera_button.text = "Stop Camera"

    def stop(self):
        self.ids.camera.play = False
        self.ids.camera_button.text = "Start Camera"

This change however will only stop the camera and freeze it on the last played frame. If we want to remove this frame on the root.stop() action and display a blank screen, we need to define this action.

In [ ]:
class CameraScreen(Screen):
    
    def start(self):
        self.ids.camera.play = True
        self.ids.camera_button.text = "Stop Camera"
        self.ids.camera.texture = self.ids.camera._camera.texture

    def stop(self):
        self.ids.camera.play = False
        self.ids.camera_button.text = "Start Camera"
        self.ids.camera.texture = None

# By adding the texture statements to both the start and stop methods, we are defining a default camera texture to the start method
# and removing the camera texture completely on the stop method, which removes the camera display altogether.

    93. L4 - Capturing a Photo from the Camera

Now we are defining the "capture" image by creating an action to save the last frame displayed on the webcam as a jpg or png file.

In [ ]:
Builder.load_string("""
<CameraScreen>:
    GridLayout:
        cols: 1
        Camera:
            id: camera
            resolution: (640, 480)
            play: False 
        Button:
            id: camera_button
            text: 'Start Camera'
            on_press: root.start() if root.ids.camera.play == False else root.stop()
        Button:
            text: 'Capture Image'
            on_press: root.capture() # By using the root keyword on press, we are rooting the frontend displays to the correct methods.

<RootWidget>:
    CameraScreen:
        id: camera_screen
        name: 'camera_screen'
""")

In [ ]:
class CameraScree(Screen):

    def capture(self):
        self.ids.camera.export_to_png()
        # This is very simple code. Simply use the "export_to_png" command already inherent to the kivy library.